In [31]:
%%writefile tcp_test.s
// --------------------------------------------------------------------------
// Bootloader para teste TCP - envia mensagens contínuas
// --------------------------------------------------------------------------

.global _start
.section .text

_start:
    // Configura stack usando movz/movk
    movz x0, #0x4100, lsl #16    // 0x41000000
    mov sp, x0
    
    // UART base usando movz/movk  
    movz x19, #0x0900, lsl #16   // 0x09000000
    mov x20, #0                  // Contador
    
    // Envia mensagem inicial
    bl send_ready
    
main_loop:
    // Incrementa contador
    add x20, x20, #1
    
    // Envia ponto a cada 10000 iterações (valor menor)
    mov x1, #10000
    udiv x2, x20, x1
    mul x3, x2, x1
    sub x4, x20, x3
    cbnz x4, main_loop
    
    // Envia um ponto
    mov w0, #'.'
    str w0, [x19]
    
    // Reset contador para evitar overflow
    mov x5, #50             // Valor menor
    cmp x2, x5
    b.lt main_loop
    
    // Envia nova linha e reinicia
    mov w0, #'\r'
    str w0, [x19]
    mov w0, #'\n'
    str w0, [x19]
    bl send_ready
    mov x20, #0
    b main_loop

send_ready:
    stp x29, x30, [sp, #-16]!
    
    mov x1, x19  // UART base
    
    // "Ready!\r\n"
    mov w0, #'R'
    str w0, [x1]
    mov w0, #'e'
    str w0, [x1]
    mov w0, #'a'
    str w0, [x1]
    mov w0, #'d'
    str w0, [x1]
    mov w0, #'y'
    str w0, [x1]
    mov w0, #'!'
    str w0, [x1]
    mov w0, #'\r'
    str w0, [x1]
    mov w0, #'\n'
    str w0, [x1]
    
    ldp x29, x30, [sp], #16
    ret

Overwriting tcp_test.s


In [32]:
%%bash
as -o tcp_test.o tcp_test.s
ld -Ttext=0x40000000 -o tcp_test.elf tcp_test.o
objcopy -O binary tcp_test.elf tcp_test.img

    qemu-system-aarch64 -machine virt -cpu cortex-a53 -nographic -kernel tcp_test.img -serial tcp::44444,server,nowait -monitor none

    nc localhost 44444

Funcionou e a saída é:

    ..................................................
    Ready!
    ..................................................
    Ready!
    ..................................................
    Ready!
    ..................................................
    Ready!
    ..................................................
    Ready!
    ..................................................
    Ready!
    ......^C
    (base) x@desk:~$ 